In [125]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  

In [126]:
df_jan = pd.read_parquet("./dataset/fhv_tripdata_2021-01.parquet")
df_feb = pd.read_parquet("./dataset/fhv_tripdata_2021-02.parquet")
df_jan.shape,df_feb.shape

((1154112, 7), (1037692, 7))

In [127]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [128]:
df_feb.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [129]:
#çompute trip duration in minutes
df_jan["duration"] = df_jan.dropOff_datetime - df_jan.pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds()/60)
df_jan

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [130]:
#Calculating the mean duration for January
df_jan.duration.mean()

19.1672240937939

In [131]:
df_jan.shape

(1154112, 8)

In [132]:
#Number of records to be removed which are not between 1 and 60mins
df_jan_removed = df_jan[(df_jan.duration<1)|(df_jan.duration >60)]
df_jan_removed.shape

(44286, 8)

In [133]:
#filter all trips which are not between 1 and 60mins
df_jan = df_jan[(df_jan.duration>=1) & (df_jan.duration <=60)]
df_jan.shape

(1109826, 8)

In [134]:
#count of missing values
df_jan.isnull().sum(axis = 0)

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
duration                        0
dtype: int64

In [135]:
#calculate the missing values
miss_val_percent = 100 * df_jan.isnull().sum() / len(df_jan)
miss_val_percent

dispatching_base_num        0.000000
pickup_datetime             0.000000
dropOff_datetime            0.000000
PUlocationID               83.527328
DOlocationID               13.327044
SR_Flag                   100.000000
Affiliated_base_number      0.069651
duration                    0.000000
dtype: float64

In [136]:
#replace the missing values with -1
df_jan = df_jan.replace(np.nan,-1)

In [137]:
PUmiss_val_percent = 100 * len(df_jan[df_jan.PUlocationID == -1.0]) / len(df_jan)
PUmiss_val_percent

83.52732770722618

In [138]:
DOmiss_val_percent = 100 * len(df_jan[df_jan.DOlocationID == -1.0]) / len(df_jan)
DOmiss_val_percent

13.327044059158823

In [139]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [140]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    #çompute trip duration in minutes
    df["duration"] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    #Select all trips less than 60
    df = df[(df.duration>=1) & (df.duration <=60)]
    
    #fill null values with -1
    df = df.replace(np.nan,-1)

    #Extract the most useful categorical and numerical variables from our dataset
    categorical = ['PUlocationID','DOlocationID']

    #Convert numerical variables to categorical variables
    df[categorical] = df[categorical].astype(str)
    
    return df

In [141]:
df_train = read_dataframe("./dataset/fhv_tripdata_2021-01.parquet")
df_val = read_dataframe("./dataset/fhv_tripdata_2021-02.parquet")
df_train.shape, df_val.shape

((1109826, 8), (990113, 8))

In [142]:
#Extract the most useful categorical variables from our dataset
categorical = ['PUlocationID','DOlocationID']

#perform data vectorization
dv = DictVectorizer()

In [143]:
#Convert our selected features to dictionaries for our vectorizers to work and fit
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [144]:
#Convert our selected features to dictionaries for our vectorizers to work
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [145]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [146]:
#Apply Linear regression on Training dataset

#train our model
lr = LinearRegression()
lr.fit(X_train, y_train)

#model prediction
y_pred = lr.predict(X_train)

#evaluate our model
rmse_train = mean_squared_error(y_train, y_pred, squared=False)
rmse_train

10.528519107206316

In [147]:
#Apply Linear regression on Validation dataset

#model prediction
y_pred = lr.predict(X_val)

#evaluate our model
rmse_val = mean_squared_error(y_val, y_pred, squared=False)

rmse_val

11.014283149347039